## Filtering and Triming the Messages

### Filtering

Import the required Libraries

In [10]:
from langchain.messages import trim_messages,AIMessage,HumanMessage,SystemMessage,AnyMessage,RemoveMessage

Creating the List of messages as a sample

In [11]:
messages: list[AnyMessage] = [
    SystemMessage(id="msg1", content="You are a helpful assistant."),
    HumanMessage(id="msg2", content="Hi, can you help me?"),
    AIMessage(id="msg3", content="Of course! What do you need?"),
    HumanMessage(id="msg4", content="Generate a summary of LangChain."),
    AIMessage(
        id="msg5", content="LangChain is a framework for building LLM applications..."
    ),
    HumanMessage(id="msg6", content="Give me an example of a chain."),
    AIMessage(id="msg7", content="Here is an example chain you can use..."),
    HumanMessage(id="msg8", content="Thanks!"),
    AIMessage(id="msg9", content="You're welcome!"),
    SystemMessage(id="msg10", content="End of conversation."),
]

For Trimming the message you can make use of the `RemoveMessage`class from the `langchain.messages`. 

In [12]:
deleted_message=[RemoveMessage(id=m.id) for m in messages[:-2]]
deleted_message

[RemoveMessage(content='', additional_kwargs={}, response_metadata={}, id='msg1'),
 RemoveMessage(content='', additional_kwargs={}, response_metadata={}, id='msg2'),
 RemoveMessage(content='', additional_kwargs={}, response_metadata={}, id='msg3'),
 RemoveMessage(content='', additional_kwargs={}, response_metadata={}, id='msg4'),
 RemoveMessage(content='', additional_kwargs={}, response_metadata={}, id='msg5'),
 RemoveMessage(content='', additional_kwargs={}, response_metadata={}, id='msg6'),
 RemoveMessage(content='', additional_kwargs={}, response_metadata={}, id='msg7'),
 RemoveMessage(content='', additional_kwargs={}, response_metadata={}, id='msg8')]

In [13]:
messages

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}, id='msg1'),
 HumanMessage(content='Hi, can you help me?', additional_kwargs={}, response_metadata={}, id='msg2'),
 AIMessage(content='Of course! What do you need?', additional_kwargs={}, response_metadata={}, id='msg3'),
 HumanMessage(content='Generate a summary of LangChain.', additional_kwargs={}, response_metadata={}, id='msg4'),
 AIMessage(content='LangChain is a framework for building LLM applications...', additional_kwargs={}, response_metadata={}, id='msg5'),
 HumanMessage(content='Give me an example of a chain.', additional_kwargs={}, response_metadata={}, id='msg6'),
 AIMessage(content='Here is an example chain you can use...', additional_kwargs={}, response_metadata={}, id='msg7'),
 HumanMessage(content='Thanks!', additional_kwargs={}, response_metadata={}, id='msg8'),
 AIMessage(content="You're welcome!", additional_kwargs={}, response_metadata={}, id='msg9'),
 SystemMessage(co

As you can see the messages is not deleted to work it wiht we need the Nodes. with `add_messages` which removes the message

Lets create a simple graph without calling LLM


In [14]:
from langgraph.graph import MessagesState, StateGraph,START,END


def filter_message(state: MessagesState) -> MessagesState:
    return {"messages": [RemoveMessage(id=m.id) for m in messages[:-2]]}

In [15]:
graph = StateGraph(MessagesState)
graph.add_node("filter",filter_message)
graph.add_edge(START,"filter")
graph.add_edge("filter",END)

agent=graph.compile()

In [16]:
agent.invoke({
    "messages":messages
})

{'messages': [AIMessage(content="You're welcome!", additional_kwargs={}, response_metadata={}, id='msg9'),
  SystemMessage(content='End of conversation.', additional_kwargs={}, response_metadata={}, id='msg10')]}

Here you get only the latest messages

### Trimming Messages

Previously we Just saw how we can filter the messages. Now we can trim the messages based on some parameter and by strategy

In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()
llm=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash"
)

Here we passed the LLM model and `max_tokens` to trimmed out the number of tokens. `strategy` available values are `first` and `last`. 

In [18]:
from langchain.messages import trim_messages

trim_messages(
    messages,
    max_tokens=20,
    token_counter=llm,
    strategy="last",
    start_on="human"   
)



[HumanMessage(content='Thanks!', additional_kwargs={}, response_metadata={}, id='msg8'),
 AIMessage(content="You're welcome!", additional_kwargs={}, response_metadata={}, id='msg9'),
 SystemMessage(content='End of conversation.', additional_kwargs={}, response_metadata={}, id='msg10')]